# Algoritmus tervezési  módszerek, programozási paradigmák I.

## (Algorithmic design techniques, programming paradigms - part I.)

## Fájlműveletek, fájlbeolvasás (File handling)

In [ ]:
filepath = "./data/P0022_names.txt"

# A felkiáltójellel Jupyterben az operációs rendszer parancsait tudjuk meghívni, 
# pl. Linux alatt a `head` parancsot",

!head -n 5 $filepath

In [ ]:
names = []

f = open(filepath, "r")
for line in f:
    names.append(line)

f.close()

In [ ]:
names[:5]

In [ ]:
names = []

f = open(filepath, "r")
for line in f:
    names.append(line.rstrip())

f.close()

In [ ]:
names[:5]

In [ ]:
f = open(filepath, "r")
names = f.read().splitlines()

f.close()

In [ ]:
names[:5]

In [ ]:
with open(filepath, "r") as f:
    names = f.read().splitlines()

In [ ]:
names[:5]

A Fibonacci-sorozattal már találkoztunk korábban, most újra megnézzük, hogy hogyan lehet kiszámolni a sorozat $n$-edik tagját.

$$
f_n = \left\{\begin{array}{ll} 1 & n = 0 \text{ vagy } n = 1 \\ f_{n-1} + f_{n-2} & n\geq 2 \end{array}\right.
$$

## Rekurzió


Rekurzív függvény esetén mindig kell egy alapeset (base-case), ami biztosítja, hogy a rekurzív függvényhívások sorozata meg fog állni. A Python nyelv lehetővé tesz a rekurzív függvényhívást, de tény, hogy a konstrukció valamelyest idegen a nyelv filozófiájától. Vannak nyelvek, ahol a rekurzió sokkal természetesebb. Az imperatív nyelvekben inkább a direkt iteráció a természetesebb, mint a rekurzió.

In [ ]:
def fibonacci(n):
    if n == 0 or n == 1:
        return 1
    
    return fibonacci(n - 1) + fibonacci(n - 2)


fibonacci(37)

Ugyanazt a függvényértéket sokszor újra és újra kiszámoljuk

```
f(5) = f(4) + f(3)
     = (f(3) + f(2)) + (f(2) + f(1))
     = [(f(2) + f(1)) + (f(1) + f(0))] + [(f(1) + f(0)) + 1]
     = [(f(1) + f(0) + 1 + (1 + 1)] + [1 + 1 + 1])
     = [1 + 1 + 3] + 3
     = 8
```

In [ ]:
nr_calls = 0

def fibonacci(n):
    global nr_calls
    nr_calls += 1
    
    if n == 0 or n == 1:
        return 1

    return fibonacci(n - 1) + fibonacci(n - 2)


_ = fibonacci(5)
print(nr_calls)

nr_calls = 0
_ = fibonacci(37)
print(nr_calls)

## Tároljuk el a már kiszámolt értékeket! (Memoization)

In [ ]:
def fibonacci(n):
    cache = {}
    
    def fib(k):
        if k == 0 or k == 1:
            return 1
        
        if k in cache:
            return cache[k]
        
        result = fib(k - 1) + fib(k - 2)
        cache[k] = result
        return result

    return fib(n)

In [ ]:
print(fibonacci(5))
print(fibonacci(37))

In [ ]:
fibonacci(1000)

In [ ]:
fibonacci(10000)

Most minden már korábban kiszámolt értéket eltárolunk, de ettől még továbbra is rekurzió amit csinálunk, annak minden hátrányával együtt. Adott $n$-re $f(n)$ értékét úgy számoljuk ki, hogy $n$ értékét csökkentve egyre kisebb értékekre próbáljuk kiszámolni $f(n)$ értékét, azaz felülről haladunk lefelé, a nagyobb $n$-ektől a kisebbek felé.

Mi lenne, ha megpróbálnánk a fordított irányban (alulról felfelé haladva) számolni? Lehetséges lenne-e meghatározni $f(0), f(1), f(2), f(3), ...$ értékét ebben a sorrendben, melynek végre eljutunk a kívánt $f(n)$ értékhez?

In [ ]:
def fibonacci(n):
    numbers = [1] * (n + 1)
    
    for k in range(2, n+1):
        numbers[k] = numbers[k-1] + numbers[k-2]
        
    return numbers[n] 


print(fibonacci(5))
print(fibonacci(37))

In [ ]:
print(fibonacci(10000))

A fenti megoldásban $f(n)$-et egyszerűen ki tudtuk számolni egy táblázatban tárolt, már korábban kiszámolt $f(n-1)$ és $f(n-2)$ értékekből. A módszer szerint az eredeti feladat egy részfeladatának eredményeit táblázatban tároljuk, majd felhasználjuk azokat az eredeti feladat megoldásának előállítására. Ezt az algoritmus-sémát, feladatmegoldási-módszert **dinamikus programozás**nak hívjuk.


Végül vegyük észre, hogy a feladat megoldásához nem szükséges eltárolni az összes Fibonacci-számot, hiszem mindig csak a legutolsó kettőre van szükség.

In [ ]:
def fibonacci(n):
    a = 1
    b = 1
    for _ in range(1, n+1):
        a, b = b, a + b

    return a


print(fibonacci(5))
print(fibonacci(37))

In [ ]:
fibonacci(10000)

Sok olyan algoritmikus feladat van, ahol különböző stratégiákkal érdemes próbálkozni. Most láttunk egy példát, hogy ugyanarra a problémára hogy néz ki több különböző megoldási stratégia.

Általában az a legjobb stratégia, amelynek a futásideje gyors és a szükséges tárhely kicsi. Ezeket a fogalmakat majd algoritmuselméleti órákon teszitek rendbe hamarosan.

## 2-SUM

Adott egy $n\leq 10^5$ szám és egy $n$ hosszú $a$ lista, melynek elemeire $-10^{5} \leq a_i \leq 10^5$ teljesül.

Keressünk olyan $1\leq p<q\leq n$ indexpárt, hogy $a[p] = −a[q]$, ha ilyen létezik.


Például:
```
5
5 4 -5 6 8
```

Ötlet: iteráljunk végig az indexpárokon és vizsgáljuk meg minden esetben, hogy teljesíti-e a feltételeket.

Ezt a megoldási módszert, amiben potenciálisan minden lehetséges jelöltet kipróbálunk, **brute force**-alapú (nyers erő) megoldásnak nevezünk. A brute force megoldás mindig működik, a kérdés inkább az, hogy vajon nem tudunk-e ennél jobbat kitalálni.

In [ ]:
def solve_2_sum(a):
    for p, elem1 in enumerate(a):
        pass

In [ ]:
def solve_2_sum(a):
    for p, elem1 in enumerate(a):
        for q, elem2 in enumerate(a[p+1:], start=p+1):
            pass

In [ ]:
def solve_2_sum(a):
    for p, elem1 in enumerate(a):
        for q, elem2 in enumerate(a[p+1:], start=p+1):
            if elem1 == -elem2:
                return p, q
            
    return None


xs = [4, 5, -3, -5, 8]
solve_2_sum(xs)

In [ ]:
import random
import time

random.seed(2112)


xs = [random.randint(-10**5, 10**5) for _ in range(50_000)]

t = time.time()
res = solve_2_sum(xs)
print(time.time() - t)

In [ ]:
xs = [1] * 50_000

t = time.time()
res = solve_2_sum(xs)
print(res)
print(time.time() - t)

**HF**: találjunk ki egy sokkal gyorsabb megoldást erre a feladatra, ami tehát egy tetszőleges, $50000$ hosszú listára legfeljebb $0.05$ másodperc alatt lefut.

**HF**: Az óra elején látott fájlban keresztnevek vannak. 

* Hány olyan lényegében különböző névpár van, hogy a pár második eleme az első névnek a megfordítottja? pl. ("NORA", "ARON"). Az ("ARON", "NORA") pár nem különbözik lényegében az előzőtől.
* A nevek kezdőbetűit tekintve melyik az 5 leggyakoribb? W-vel vagy C-vel kezdődik több név?

## Programozási paradigmák

Programozási paradigma alatt programozási stílust, gondolkodási sémát értünk, amit az adott nyelv feature-készlete diktál. A leggyakoribb paradigmák az alábbi csoportokba sorolhatók, de ennél sokkal finomabb felosztás is elképzelhető:

* imperatív
    * procedurális
    * objektum-orientált
    
* deklaratív
    * funkcionális
    * logikai
    * reaktív

Az **imperatív** paradigma fő eleme az utasítás, ahol a gépet utasítjuk, hogy milyen parancsokat hajtson végre. Az ilyen nyelvek főbb jellemzői, alapvető fogalmai: változók, utasítások (statement), ciklusok.

Ezzel szemben áll a **deklaratív** stílus, amikor egy adott feladat megoldásakor olyan kódot írunk, ami a megoldandó feladat egyfajta leírását tartalmazza és nem a konkrét megvalósítás részleteit. Arra fókuszálunk, hogy mit szeretnénk elérni és nem arra, hogy hogyan.

A modern nyelvek általában több paradigmát támogatnak, egyeseket jobban, másokat kevésbé.

A jelenleg legnépszerűbb programozási nyelvek általában az imperatív (procedurális) és az objektum-orientált paradigmát követik (C++, Java, C#), azonban az utóbbi évtizedben erősödik a funkcionális paradigma is. 

A Python is ezeket a paradigmákat követi, ezeket engedi a felhasználóinak, de nem egyforma mértékben. A Python elsősorban egy imperatív (deklaratív) nyelv, azaz függvényekbe szervezett utasításokat használ, ugyanakkor objektum-orientált is (Pythonban minden objektum, amint ezt már említettük). Emellett, a nyelvtől némileg idegen módon támogat bizonyos elemeket a funkcionális paradigmából is.

A következőkben a Python nyelv eddig nem tárgyalt programozási paradigmáiról lesz szó.

## Objektum-orientált programozás / paradigma (OOP)

Ez a paradigma az ún. objektumok köré szerveződik, amelyek olyan tárolóegységek, melyek egyszerre tartalmaznak adatokat (melyeket mezőknek, vagy attribútumoknak nevezünk), illetve kódot, függvényeket, ami ezen adatok manipulására szolgál. Ezek neve metódus. Metódusokkal elérhető, hogy az objektum adatait magában az objektumban módosítsuk, így az objektum maga egy állapotot tart fenn (pl. egy bankszámla aktuális állapota).

Az objektum-orientált programozásban ilyen objektumok épülnek fel a futó programban, melyek egymással interakcióban vannak. A legtöbb OO-nyelv alapfogalma az osztály (class), ekkor az objektumok ezen osztályok példányai (instance).

Pythonban az OOP nincs előtérben abban az értelemben, hogy lehet Python kódot írni osztályok nélkül is, mindössze függvényekkel operálva. Bizonyos értelemben nem a Python a legmegfelelőbb nyelv, ha igazán meg akarjuk érteni az OOP fő elveit.

Más OOP nyelvekben (pl. Java-ban) sokkal jobban előtérben vannak ezek a fogalmak és nélkülük egyszerűen nem is lehet kódot írni. Ennek ellenére mégis hasznos megismerni az OOP alapjait, mert sok esetben hasznos lehet létező kód megértésében, illetve közepes vagy nagyobb méretű kódbázis fejlesztésénél.

Az első OOP-nyelv a Smalltalk volt (1972), és a fogalmak fejlődése során némileg átalakulva az OOP paradigma az 1990-es és korai 2000-es évek uralkodó paradigmájává vált.

In [ ]:
# A legegyszerűbb osztály

class Person:
    pass


p = Person()
print(p)
print(type(p))

In [ ]:
# Person egy osztály

class Person:
    pass


# p a Person osztály egy példánya
p = Person()

# a p példányhoz dinamikusan lehet attribútumokat adni
p.name = "Ann"
p.age = 25

print(p)

print(p.name)
print(p.age)

print(p.__dict__)

In [ ]:
# Az __init__ metódus inicializálja az osztály, 
# így az attribútúmok nem utólag lesznek hozzáadva az adott példányhoz  

class Person:
    # Minden metódus első paramétere maga az osztálypéldány
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    
p = Person("Ann", 25) 
print(p)
print(p.name)
print(p.age)

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def introduction(self):
        return f"Hi, my name is {self.name}!"
    
    
p = Person("Ann", 25)

p.introduction()

In [ ]:
print(p)

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"
        
    def introduction(self):
        return f"Hi, my name is {self.name}!"
    
    
p = Person("Ann", 25)
print(p)

Van egy `__str__` metódus is a `__repr__` mellett. Az utóbbi általában magunknak szól, az előbbi a végfelhasználónak. Ha osztályt írunk, `__repr__` metódust mindig érdemes implementálni.

OOP-ből ismert fogalmak lehetnek a `public`, `protected` és `private` attribútumok fogalma. Pythonban nincs kulcsszó ezekre, egyszerűen egy nevezéktani konvenció biztosítja, hogy melyik attribútum érhető el kívülről, és melyeket szeretnénk protected-nek vagy privátnak tekinteni.

Más nyelvekből ismerős getter és setter függvényeket lehet ugyan írni, de ne felejtsük, hogy ez itt Python, nincs kikényszerítve, hogy ilyenek írjunk és nem is feltétlenül szokás.

In [ ]:
class Person:
    class_variable = 123
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def __repr__(self):
        return f"Person(name={self.name}, age={self.age})"

    def is_pensioner(self):
        return self.age >= 65
    
    
p = Person(name="Ann", age=25)  

print(p.is_pensioner())
print(p.class_variable)
print(Person.class_variable)

Miért használunk osztályokat? Először is, osztályokat használni nem kötelező, de előbb-utóbb találkoztok velük, mások kódjaiban, vagy könyvtárakból való importok esetén. Néhány érv a használatuk mellett:

* Valamilyen módon összetartozó adatokat akarunk eltárolni egységbe zárva (encapsulation)
* Egy állapotot kell tartani, illetve ezt az állapotot kell tudnunk megfelelő módon változtatni
* Az adatot csak meghatározott módon lehessen manipulálni, a felhasználónak nem kell tudnia, hogy mi a belső implementáció (absztrakció)
* Hierarchiába szervezhető adattípusok vannak (öröklődés)

In [ ]:
class BankAccount:
    def __init__(self, account_id, initial_amount=0):
        self._id = account_id
        self._amount = initial_amount
        
    def __repr__(self):
        return f"BankAccount(id={self._id}, amount={self._amount})."
    
    def amount(self):
        return self._amount
        
    def deposit(self, money):
        self._amount += money
        
    def withdraw(self, money):
        if self._amount >= money:
            self._amount -= money
            return money
        
        # Kivétel dobása helyett ilyenkor akár 0 fabatkát is visszadhatnánk,
        # az egyenleget változatlanul hagyva
        raise Exception("Not enough money on your account.")

In [ ]:
account = BankAccount(123)

account.deposit(100)
account.deposit(200)
_ = account.withdraw(250)
print(account)
print(account.amount())

Ez a számla nem túl biztonságos, mert az `_amount` mezőt lehet kívülről manipulálni, csak a konvención és az én jóindulatomon múlt, hogy titokban ne adjak pénzt közvetlenül ehhez a mezőhöz.

In [ ]:
account = BankAccount(123)
account._amount = 10000

print(account.amount())

Pythonban lehet ennél szigorúbb hozzáférést adni a tényleg privátnak gondolt mezőkhöz, illetve megfelelő settereket és gettereket itt is be lehet állítani, de mi most ennél tovább nem megyünk.

In [ ]:
class BankAccount:
    def __init__(self, account_id, initial_amount=0):
        self._id = account_id
        self._amount = initial_amount
        
    def __repr__(self):
        return f"BankAccount(id={self._id}, amount={self._amount})."
    
    @property
    def amount(self):
        return self._amount
        
    def deposit(self, money):
        self._amount += money
        
    def withdraw(self, money):
        if self._amount >= money:
            self._amount -= money
            return money
        
        raise Exception("Not enough money on your account.")

In [ ]:
account = BankAccount(123)
account.deposit(100)

print(account)
print(account.amount)

In [ ]:
account.amount = 1000